In [29]:
import os
import pickle

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras import models
# import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [30]:
# RTX On
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [31]:
def load_data(file):
    with open(file, 'rb') as fo:
        try:
            samples = pickle.load(fo)
        except UnicodeDecodeError:  # python 3.x
            fo.seek(0)
            samples = pickle.load(fo, encoding='latin1')

    data, labels = samples['data'], samples['labels']

    data = np.array(data, dtype=np.float32) / 255
    labels = np.array(labels, dtype=np.int32)
    return data, labels

In [32]:
def make_model(num_ch_c1, num_ch_c2, use_dropout):

    model = tf.keras.Sequential()
    
    if (use_dropout == False):
        # Input Layer
        model.add(layers.Input(shape=(3072,)))
        model.add(layers.Reshape(target_shape=(32, 32, 3), input_shape=(3072,)))
        # C₁ Convolutional Layer
        model.add(layers.Conv2D(num_ch_c1, 9, padding='valid', activation='relu', input_shape=(None, None, 3)))
        # S₁ Max Pooling Layer
        model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2), padding='valid'))
        # C₂ Convolutional Layer
        model.add(layers.Conv2D(num_ch_c2, 5, padding='valid', activation='relu', input_shape=(None, None, 3)))
        # S₂ Max Pooling Layer
        model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2), padding='valid'))
        # Flatten
        model.add(layers.Flatten())
        # F₃ Fully-Connected Layer
        model.add(layers.Dense(300, use_bias=True))  # Here no softmax because we have combined it with the loss   
        # F₄ Fully-Connected Layer 
        model.add(layers.Dense(10, use_bias=True, input_shape=(10,)))  # Here no softmax because we have combined it with the loss
    else:
        # Input Layer
        model.add(layers.Input(shape=(3072,)))
        model.add(layers.Reshape(target_shape=(32, 32, 3), input_shape=(3072,)))
        # C₁ Convolutional Layer
        model.add(layers.Conv2D(num_ch_c1, 9, padding='valid', activation='relu', input_shape=(None, None, 3)))
        # S₁ Max Pooling Layer
        model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2), padding='valid'))
        # C₂ Convolutional Layer
        model.add(layers.Conv2D(num_ch_c2, 5, padding='valid', activation='relu', input_shape=(None, None, 3)))
        # S₂ Max Pooling Layer
        model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2), padding='valid'))
        # Flatten
        model.add(layers.Flatten())
        # F₃ Fully-Connected Layer
        model.add(layers.Dense(300, use_bias=True))  # Here no softmax because we have combined it with the loss   
        # D₁ Dropout Layer
        model.add(layers.Dropout(0.5))
        # F₄ Fully-Connected Layer 
        model.add(layers.Dense(10, use_bias=True, input_shape=(10,)))  # Here no softmax because we have combined it with the loss
        # D₂ Dropout Layer
        model.add(layers.Dropout(0.5))

    return model

In [33]:
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

# Enabled, as we have decided the optimal channel values
num_ch_c1 = 90  # Question 2
num_ch_c2 = 100  # Question 2

epochs = 1000  # Fixed
batch_size = 128  # Fixed
learning_rate = 0.001

optimizer_ = 'Adam'  # Question 3
#use_dropout = False  # Question 3(d) (see make_model)

In [34]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [35]:
if optimizer_ == 'SGD':
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
elif optimizer_ == 'SGD-momentum':  # Question 3(a)
    momentum = 0.1
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum)
elif optimizer_ == 'RMSprop':  # Question 3(b)
    optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate)
elif optimizer_ == 'Adam':  # Question 3(c)
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
else:
    raise NotImplementedError(f'You do not need to handle [{optimizer_}] in this project.')

In [ ]:
'''
Load Training and Test Datasets
'''
x_train, y_train = load_data('/content/gdrive/My Drive/Colab Notebooks/data_batch_1')
x_test, y_test = load_data('/content/gdrive/My Drive/Colab Notebooks/test_batch_trim')

In [ ]:
# Create folder to store one more level down
if not os.path.exists(f'/content/gdrive/My Drive/Colab Output/models/{num_ch_c1}_{num_ch_c2}_{optimizer_}_dropout'):
    os.mkdir(f'/content/gdrive/My Drive/Colab Output/models/{num_ch_c1}_{num_ch_c2}_{optimizer_}_dropout')
if not os.path.exists(f'/content/gdrive/My Drive/Colab Output/models/{num_ch_c1}_{num_ch_c2}_{optimizer_}_no_dropout'):
    os.mkdir(f'/content/gdrive/My Drive/Colab Output/models/{num_ch_c1}_{num_ch_c2}_{optimizer_}_no_dropout')

In [36]:
# Set Question Variable for file purposes
question = 'q3c'

In [ ]:
'''
Train Model for both without and with dropout
'''
# Optimal Values Chosen, c1 = 90, c2 = 100
dropout_list = [True, False]

for each in dropout_list:

    use_dropout = each
    model = make_model(num_ch_c1, num_ch_c2, use_dropout) 

    # Train RMSprop
    model.compile(optimizer=optimizer, loss=loss, metrics='accuracy')
    history = model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(x_test, y_test),
        verbose=0)

    # Extract Loss Values
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    # Extract Accuracy Values
    train_acc = history.history['accuracy']
    test_acc = history.history['val_accuracy']

    # Save instanced variables to list
    instanced_variables = [train_loss, val_loss, train_acc, test_acc]

    # Save model
    if use_dropout:
        model.save(f'/content/gdrive/My Drive/Colab Output/models/{num_ch_c1}_{num_ch_c2}_{optimizer_}_dropout')
        filename = f'/content/gdrive/My Drive/Colab Output/part_a_{question}_{num_ch_c1}_{num_ch_c2}_{optimizer_}_dropout_variables'
        outfile = open(filename,'wb')
        pickle.dump(instanced_variables, outfile)
        outfile.close()    
    else:
        model.save(f'/content/gdrive/My Drive/Colab Output/models/{num_ch_c1}_{num_ch_c2}_{optimizer_}_no_dropout')
        filename = f'/content/gdrive/My Drive/Colab Output/part_a_{question}_{num_ch_c1}_{num_ch_c2}_{optimizer_}_no_dropout_variables'
        outfile = open(filename,'wb')
        pickle.dump(instanced_variables, outfile)
        outfile.close()    

In [37]:
# Retrieve previously saved DROPOUT instanced variables w/ pickle rick
infile = open(f'/content/gdrive/My Drive/Colab Output/part_a_{question}_{num_ch_c1}_{num_ch_c2}_{optimizer_}_dropout_variables', 'rb')
retrieved_variables_dropout = pickle.load(infile)
infile.close()

# Retrieve previously saved NO DROPOUT instanced variables w/ pickle rick
infile = open(f'/content/gdrive/My Drive/Colab Output/part_a_{question}_{num_ch_c1}_{num_ch_c2}_{optimizer_}_no_dropout_variables', 'rb')
retrieved_variables_no_dropout = pickle.load(infile)
infile.close()

# Load retrieved variables into instance variables for plotting
train_loss_dropout, val_loss_dropout, train_acc_dropout, val_acc_dropout = retrieved_variables_dropout
train_loss_no_dropout, val_loss_no_dropout, train_acc_no_dropout, val_acc_no_dropout = retrieved_variables_no_dropout

In [38]:
# get highest accuracy values
highest_acc = max(val_acc_no_dropout)

# get average accuracy
average_acc = sum(val_acc_no_dropout)/len(val_acc_no_dropout)

print('Highest Acc')
print(highest_acc)

print('')

print('Avg Acc')
print(average_acc)

Highest Acc
0.10000000149011612

Avg Acc
0.10000000149011612


In [ ]:
dropout_list = [True, False]
for each in dropout_list:
    
    use_dropout = each
    
    if (use_dropout == True):
        # using dropout
        # Save the plot for loss
        plt.plot(range(1, len(train_loss_dropout) + 1), train_loss_dropout, label='Train')
        plt.plot(range(1, len(val_loss_dropout) + 1), val_loss_dropout, label='Test')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend()
        plt.title(f'{optimizer_} Model Loss with dropout')
        plt.savefig(f'/content/gdrive/My Drive/Colab Output/results/part_a_{question}_{num_ch_c1}_{num_ch_c2}_{optimizer_}_dropout_loss.pdf')
        plt.savefig(f'/content/gdrive/My Drive/Colab Output/results/part_a_{question}_{num_ch_c1}_{num_ch_c2}_{optimizer_}_dropout_loss.png')
        plt.close()
        # Save the plot for accuracy (with dropout)
        plt.plot(range(1, len(train_acc_dropout) + 1), train_acc_dropout, label='Train')
        plt.plot(range(1, len(val_acc_dropout) + 1), val_acc_dropout, label='Test')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend()
        plt.title(f'{optimizer_} Model Accuracy with dropout')
        plt.savefig(f'/content/gdrive/My Drive/Colab Output/results/part_a_{question}_{num_ch_c1}_{num_ch_c2}_{optimizer_}_dropout_accuracy.pdf')
        plt.savefig(f'/content/gdrive/My Drive/Colab Output/results/part_a_{question}_{num_ch_c1}_{num_ch_c2}_{optimizer_}_dropout_accuracy.png')
        plt.close()
    else:
        # no dropout
        # Save the plot for loss
        plt.plot(range(1, len(train_loss_no_dropout) + 1), train_loss_no_dropout, label='Train')
        plt.plot(range(1, len(val_loss_no_dropout) + 1), val_loss_no_dropout, label='Test')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend()
        plt.title(f'{optimizer_} Model Loss no dropout')
        plt.savefig(f'/content/gdrive/My Drive/Colab Output/results/part_a_{question}_{num_ch_c1}_{num_ch_c2}_{optimizer_}_no_dropout_loss.pdf')
        plt.savefig(f'/content/gdrive/My Drive/Colab Output/results/part_a_{question}_{num_ch_c1}_{num_ch_c2}_{optimizer_}_no_dropout_loss.png')
        plt.close()
        # Save the plot for accuracy (with dropout)
        plt.plot(range(1, len(train_acc_no_dropout) + 1), train_acc_no_dropout, label='Train')
        plt.plot(range(1, len(val_acc_no_dropout) + 1), val_acc_no_dropout, label='Test')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend()
        plt.title(f'{optimizer_} Model Accuracy no dropout')
        plt.savefig(f'/content/gdrive/My Drive/Colab Output/results/part_a_{question}_{num_ch_c1}_{num_ch_c2}_{optimizer_}_no_dropout_accuracy.pdf')
        plt.savefig(f'/content/gdrive/My Drive/Colab Output/results/part_a_{question}_{num_ch_c1}_{num_ch_c2}_{optimizer_}_no_dropout_accuracy.png')
        plt.close()